# Importing Libraries

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

In [2]:
def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&crid=1YOA34FVDWY3Y'
    search_term = search_term.replace(' ', '+')
    url = template.format(search_term)
    url += '&page={}'
    return url

def extract_record(item):
    try:
        atag = item.h2.a
        description = atag.text.strip()
        url = 'https://www.amazon.com' + atag.get('href')
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError as e:
        print("Error while extracting record:", e)
        return

    return (description, price, rating, review_count, url)

def fetch_data_from_amazon(driver, url):
    records = []
    for page in range(1, 21):
        print(f"Fetching page {page} ...")
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    return records

def save_to_csv(records, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)
    print(f"Data saved successfully to {filename}.")

def main():
    search_term = input("Enter the product you want to search for: ")
    filename = input("Enter the name for the CSV file (e.g., output.csv): ")

    try:
        url = get_url(search_term)
        with webdriver.Chrome() as driver:
            records = fetch_data_from_amazon(driver, url)
        save_to_csv(records, filename)

    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
if __name__ == "__main__":
    main()

Fetching page 1 ...
Fetching page 2 ...
Fetching page 3 ...
Fetching page 4 ...
Error while extracting record: 'NoneType' object has no attribute 'find'
Fetching page 5 ...
Fetching page 6 ...
Fetching page 7 ...
Fetching page 8 ...
Fetching page 9 ...
Fetching page 10 ...
Fetching page 11 ...
Fetching page 12 ...
Fetching page 13 ...
Fetching page 14 ...
Fetching page 15 ...
Fetching page 16 ...
Fetching page 17 ...
Fetching page 18 ...
Fetching page 19 ...
Fetching page 20 ...
Data saved successfully to amazon_protein_081823.csv.
